## 今天的作業
在鐵達尼資料集中，以Titanic_train.csv 中，首先將有遺失值的數值刪除，試著將課堂中所學的方法應用上去。
* Q1: 目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。


In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
print(pd.__version__)
%matplotlib inline




# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

 

0.23.1
1.0.5


## 讀入資料

In [3]:
df_train = pd.read_csv("./Titanic_train.csv")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
#1.產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態      
  
df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB


In [6]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived_cate
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [11]:
df_train['Name'].value_counts()

Harper, Miss. Annie Jessie "Nina"            1
Chapman, Mr. John Henry                      1
Ringhini, Mr. Sante                          1
Moen, Mr. Sigurd Hansen                      1
Peter, Mrs. Catherine (Catherine Rizk)       1
                                            ..
Karun, Miss. Manca                           1
Murphy, Miss. Katherine "Kate"               1
Endres, Miss. Caroline Louise                1
Yasbeck, Mr. Antoni                          1
Mellinger, Mrs. (Elizabeth Anne Maidment)    1
Name: Name, Length: 891, dtype: int64

In [12]:
df_train['Ticket'].value_counts()

CA. 2343        7
347082          7
1601            7
CA 2144         6
3101295         6
               ..
F.C.C. 13531    1
234686          1
A/5. 3337       1
345777          1
A4. 54510       1
Name: Ticket, Length: 681, dtype: int64

In [13]:
df_train['PassengerId'].value_counts()

891    1
293    1
304    1
303    1
302    1
      ..
591    1
590    1
589    1
588    1
1      1
Name: PassengerId, Length: 891, dtype: int64

In [18]:
#我們先把遺失值刪除

complete_data=df_train.dropna()

#先刪除是獨特資料的欄位，先檢視其他變數
complete_data=complete_data.drop(['Name','Ticket','PassengerId'], axis=1)
display(complete_data.head())

print(complete_data.shape)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate
1,1,1,female,38.0,1,0,71.2833,C85,C,1
3,1,1,female,35.0,1,0,53.1000,C123,S,1
6,0,1,male,54.0,0,0,51.8625,E46,S,0
10,1,3,female,4.0,1,1,16.7000,G6,S,1
11,1,1,female,58.0,0,0,26.5500,C103,S,1


(183, 10)


In [19]:
#取出數值型特徵

num_features = []

for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

6 Numeric Features : ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [20]:
#取出類別型特徵

cat_features = []

for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'object':
        cat_features.append(feature)
print(f'{len(cat_features)} category Features : {cat_features}\n')

4 category Features : ['Sex', 'Cabin', 'Embarked', 'Survived_cate']



## 定義 x,y

In [31]:
x=complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']] #Cabin有太多遺失值
y=complete_data['Survived']
display(x.head())
display(y.head())

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,female,C
3,1,35.0,1,0,53.1000,female,S
6,1,54.0,0,0,51.8625,male,S
10,3,4.0,1,1,16.7000,female,S
11,1,58.0,0,0,26.5500,female,S


1     1
3     1
6     0
10    1
11    1
Name: Survived, dtype: int64

### Q1: 目標變數為 Survived，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性!


In [32]:
#離散要轉換成數值，['Sex',  'Cabin', 'Embarked']
#Can only use .cat accessor with a 'category' dtype
# .cat.codes ：分類的編碼，用數字表示分類數據

x1 = x.copy()
c = x1.loc[:,'Sex'].astype('category')
d = dict(enumerate(c.cat.categories))
print (d)
x1.loc[:,'Sex']=x1.loc[:,'Sex'].astype('category').cat.codes # ok
display(x1.head(5))

{0: 'female', 1: 'male'}


,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,0,C
3,1,35.0,1,0,53.1000,0,S
6,1,54.0,0,0,51.8625,1,S
10,3,4.0,1,1,16.7000,0,S
11,1,58.0,0,0,26.5500,0,S


In [34]:
from sklearn import preprocessing

x2 = x.copy()

le = preprocessing.LabelEncoder()

x2['Sex'] = le.fit_transform(x2['Sex'])

x2.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,0,C
3,1,35.0,1,0,53.1000,0,S
6,1,54.0,0,0,51.8625,1,S
10,3,4.0,1,1,16.7000,0,S
11,1,58.0,0,0,26.5500,0,S


In [35]:
le = preprocessing.LabelEncoder()

x2['Embarked'] = le.fit_transform(x2['Embarked'])

x2.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,0,0
3,1,35.0,1,0,53.1000,0,2
6,1,54.0,0,0,51.8625,1,2
10,3,4.0,1,1,16.7000,0,2
11,1,58.0,0,0,26.5500,0,2


In [37]:
#執行 包裝法

estimator = SVC(kernel="linear")

selector = RFE(estimator, n_features_to_select=1, step=1)

selector = selector.fit(x2, y)
selector

RFE(estimator=SVC(kernel='linear'), n_features_to_select=1)

In [38]:
## True= selected feature
print(selector.support_)

[False False False False False  True False]


In [40]:
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)

[3 5 2 6 7 1 4]


In [42]:
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)

['Sex']


Pclass-Age-SibSp-Parch-Fare-Sex-Embarked 的依序排序為 [3 5 2 6 7 1 4]